In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.optim import Adam
import torch.nn as nn

# Preprocess

In [6]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5]
                        )
])

# Dataloader

In [9]:
trainPath="c:\\Users\\admin\\Documents\\PPKE-ITK\\7_szemeszter\\Neurhalok\\Neurhalok_competition\\ppke-itk-neural-networks-2022-challenge\\db_chlorella_renamed_TRAIN_merged"

In [10]:
train = DataLoader(ImageFolder(trainPath, transformer), shuffle=True)

# Network

In [60]:
class DummyNet(nn.Module):
    def __init__(self, classes=8):
        super(DummyNet,self).__init__()
        
        self.conv = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.batch= nn.BatchNorm2d(num_features=8)
        self.leaky= nn.LeakyReLU()
        self.lin1=nn.Linear(in_features=127008, out_features=16)
        self.lin2=nn.Linear(in_features=16, out_features=8)
        self.flat=nn.Flatten()

    def forward(self, input):
        output = self.conv(input)
        output = self.batch(output)
        output = self.leaky(output)
        
        output = self.flat(output)
        output = self.lin1(output)
        output = self.lin2(output)
        
        return output


In [61]:
device = torch.device('cpu')
model = DummyNet(8).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [67]:
epoch_num = 2

# Train

In [68]:
for epoch in range(epoch_num):
    model.train()
    train_loss = 0.0
    for i, (images, label) in enumerate(train):
        optimizer.zero_grad()
        output = model(images)
        loss = lossfn(output,label)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().data * images.size(0)
    train_loss=train_loss/ len(train.dataset)
    
    print("epoch done: ",epoch," loss: ",train_loss.item())

epoch done:  0  loss:  tensor(0.5027)
epoch done:  1  loss:  tensor(0.1184)



# Eval

In [71]:
valid_loss= 0.0
for i, (images, label) in enumerate(train):
    model.eval()
    output = model(images)
    loss = lossfn(output,label)
    valid_loss += loss.cpu().data * images.size(0)
valid_loss=valid_loss/len(train.dataset)
print(f"valid_loss: {valid_loss.item()}")

valid_loss: 0.6228397488594055
